In [1]:
from graphtorch import SparseMatrix, SparseModel
import numpy as np
import torch 
import torch.nn as nn
import random
import graphtorch

__Example 2__ 

In [16]:
mat2 = np.array([[2,0,0,0,0,0],
                [0,0,0,3,0,1]])
in_dim = 5
out_dim = 1
mat_wann2 = SparseMatrix(mat2, in_dim, out_dim)

In [5]:
mat_wann2.connection_count

3

In [67]:
mat_wann2.hidden_dim

[1]

__Example 1__ 

In [38]:
mat1 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])  
in_dim = 5   
out_dim = 2  
mat_wann1 = SparseMatrix(mat1, in_dim, out_dim)

In [10]:
print(mat_wann1.connection_count)
print(mat_wann1.hidden_dim)

12
[3, 2]


__three methods to change architecture__

In [4]:
def change_activation(SparseMatrix, activations):
    
    in_dim = SparseMatrix.in_dim
    out_dim = SparseMatrix.out_dim
    mat = SparseMatrix.mat  
    indices = (mat != 0) #0이 아닌 index들만 activation 바꿀 수 있으니까 true false matrix 받기   
    possible = [] # activation을 바꿀 수 있는 index 리스트. [[0, 0], [1, 3], [1, 5]] 이런 식으로 나옴 
    
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            if indices[i,j] == True:
                possible += [[i,j]]
    
    ran_idx = random.choice(possible) # [1,5] 이런 식으로 나오는데     
    
    #activations = [1, 2, 3] #linear, relu, sigmoid   
    ran_activation = random.choice(activations)
    
    #change activation
    mat[ran_idx[0],ran_idx[1]] = ran_activation 
    
    new_SparseMatrix = graphtorch.SparseMatrix(mat, in_dim, out_dim) #이렇게 해야 callable 하더라고.. 
    

    return new_SparseMatrix    

In [7]:
def add_connection(SparseMatrix):
    
    mat = SparseMatrix.mat 
    in_dim = SparseMatrix.in_dim
    out_dim = SparseMatrix.out_dim  
    hidden_dim = SparseMatrix.hidden_dim 
    
    #random 하게 connection 추가 
    # 같은 layer끼리는 connection x -> hidden layer끼리만 봐주면 됨 (input끼리 output끼리는 matrix에 없으니까) 
    # 0인 element 중에서 임의로 하나 바꾸기 -> 1,2,3 중에서(linear, relu, sigmoid)   
    # 가능한 element 중에서 랜덤하게 하나의 element 뽑고, 거기서 또 (1,2,3) 중에서 랜덤하게 하나 뽑기  
    
    ###################
    #hidden layer끼리 연결되어 있는 indices 들 뽑아내기
    hidden_wise_indices = []
    #print(hidden_dim[0])
    row_start = 0
    row_end = 0 + hidden_dim[0] - 1
    col_start = in_dim 
    col_end = in_dim + hidden_dim[0] - 1 
    #print(col_start, col_end)
    
    for hidden_count, current_hidden_dim in enumerate(hidden_dim): 
        #이전 laye의 dimension
        for i in range(row_start, row_end + 1):
            #print(col_start, col_end)
            for j in range(col_start, col_end + 1):
                
                hidden_wise_indices += [[i,j]]
        
        row_start += current_hidden_dim  
        col_start += current_hidden_dim 
        row_end += (current_hidden_dim - 1)  
        col_end += (current_hidden_dim - 1)

    ##################################    
    #우선 0인 element들의 indices 받아오기   
    indices = (mat == 0)     
    zero_indices = [] # 0이 아닌 애들 indexes 우선 받아오기 
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            if indices[i,j] == True:
                zero_indices += [[i,j]]
    
    
    #################################
    # hidden_wise_indices에 해당되지 않는 zero_indices만이 possible에 포함될 수 있음  
    # (zero_indices) and (! hidden_wise_indices)
    # 같은 hidden layer끼리 있지 않은 인덱스들 
    possible = []
    possible = [value for value in zero_indices if value not in hidden_wise_indices]  
    
    
    # 우선 랜덤하게 어떤 element 바꿀 지 선택
    ran_idx = random.choice(possible)
    # select random activation -> 안하고 바로 liear 로 세팅
    #possible_activation = [1, 2, 3] #linear, relu, sigmoid   
    #ran_activation = random.choice(possible_activation)
    
    mat[ran_idx[0], ran_idx[1]] = 1   
    
    new_SparseMatrix = graphtorch.SparseMatrix(mat, in_dim, out_dim)  
    
    
    
    return new_SparseMatrix
            
    

In [12]:
def add_node(SparseMatrix, activations):
    
    mat = SparseMatrix.mat 
    in_dim = SparseMatrix.in_dim
    out_dim = SparseMatrix.out_dim  
    hidden_dim = SparseMatrix.hidden_dim 
    
    #이미 connection이 존재하는 부분 사이에 node 추가하기 
    #원래 connection에 존재하던 activation 앞으로 넘길지 뒤로 넘길지 정하고, 나머지에는 랜덤하게 정해주기
    #노드 추가해주면 dimension 추가해주고 connection 다시 계산해 줘야 하는데.. 
    
    # connection이 존재하는 indices추출 
    indices = (mat != 0)
    possible = [] #node를 add할 수 있는 connection들 index 리스트 
    for i in range(indices.shape[0]):
        for j in range(indices.shape[1]):
            if indices[i,j] == True:
                possible += [[i,j]]    
    #가능한 인덱스 중에서 하나 선택 
    ran_idx = random.choice(possible) #[i,j] 형태로 나옴 
    #랜덤하게 정할 activation 뽑기  
    #possible_activation = [1,2,3]
    ran_activation = random.choice(activations) 
    #원래 connection에 존재하던 activation type 앞으로 할지 뒤로 할지
    front = 0 
    back = 1
    front_or_back = random.choice([front, back]) 
    #선택된 index의 원래 actavation 가지고있기
    original_activation = mat[ran_idx[0], ran_idx[1]] 
    #선택된 index의 element 값 0으로 만들어주기
    mat[ran_idx[0], ran_idx[1]] = 0
    
    
    ## 몇번째 layer에 해당하는지만   
    #그 layer 마지막에 추가해주기   
    total_dim = [in_dim]
    for i in range(len(hidden_dim)):
        total_dim += [hidden_dim[i]]
    total_dim += [out_dim]  
    
    
    
    #선택된 index 사이에 추가할 노드가 몇번째 hidden layer의 몇번째 hidden node인지 알아내야 -> mat의 index로 나오도록 
    # 1) from이 input이고 to가 output인 경우
        #
    
    # 2) from이 input이고 to가 첫번째 hidden 인 경우
        # 삽입되는 노드가 to에 해당하는 hidden node 바로 
    # 3) from이 hidden이고 to가 output인 경우
    # 4) from이 hidden이고 to가 hidden인 경우 
    
    
    
    #matrix dimension 늘리기(앞의 index 활용해서 늘려야함)  
    
    
    #element 값 할당하기 
    
    
    
    

__print 디버깅__ 

In [11]:
new = add_connection(mat_wann1)   
print(new.mat)

[[0 2 0 0 2 0 0 0 0 0]
 [2 0 2 0 0 0 0 0 0 0]
 [0 2 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 1 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 3]
 [0 0 0 0 0 0 0 0 3 0]]


In [109]:
indices = (mat_wann1.mat == 0)
possible = []
for i in range(indices.shape[0]):
    for j in range(indices.shape[1]):
        if indices[i,j] == True:
            possible += [[i,j]]

            
print(possible)

[[0, 0], [0, 2], [0, 3], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 1], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [2, 0], [2, 2], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 7], [3, 8], [3, 9], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 8], [4, 9], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 9]]


In [23]:
possible_activation = [1,2,3]
random.choice(possible_activation)

1

In [61]:
sparsemat = change_activation(mat_wann2)
print(sparsemat.mat)
print(type(sparsemat))

[[2 0 0 0 0 0]
 [0 0 0 2 0 1]]
<class 'graphtorch.SparseMatrix'>


In [130]:
hidden_wise_indices = []
row_start = 0
row_end = 0 + mat_wann1.hidden_dim[0] - 1
col_start = in_dim 
col_end = in_dim + mat_wann1.hidden_dim[0] - 1 
    
for hidden_count, current_hidden_dim in enumerate(mat_wann1.hidden_dim): 
    
    #이전 laye의 dimension
    
    print("curent_hidden_dim: {}".format(current_hidden_dim))
    
    print("row_start: {}".format(row_start))
    print("row_end: {}".format(row_end))
    print("col_start: {}".format(col_start))
    print("col_end: {}".format(col_end))
    for i in range(row_start, row_end + 1):
        for j in range(col_start, col_end + 1):
            hidden_wise_indices += [[i,j]]
    
    row_start += current_hidden_dim  
    col_start += current_hidden_dim 
    row_end += (current_hidden_dim - 1)  
    col_end += (current_hidden_dim - 1) 
    

curent_hidden_dim: 3
row_start: 0
row_end: 2
col_start: 5
col_end: 7
curent_hidden_dim: 2
row_start: 3
row_end: 4
col_start: 8
col_end: 9


In [102]:
print(hidden_wise_indices)

[[0, 5], [0, 6], [0, 7], [1, 5], [1, 6], [1, 7], [2, 5], [2, 6], [2, 7], [3, 8], [3, 9], [4, 8], [4, 9]]


In [124]:
lists = [value for value in possible if value not in hidden_wise_indices ]
print(lists)

[[0, 0], [0, 2], [0, 3], [0, 8], [0, 9], [1, 1], [1, 3], [1, 4], [1, 8], [1, 9], [2, 0], [2, 2], [2, 4], [2, 8], [2, 9], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 7], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 9]]


In [18]:
#### test add dimension
print(mat2)
print(mat2.shape)
idx = [1,3] 
idx

[[2 0 0 0 0 0]
 [0 0 0 3 0 1]]
(2, 6)


[1, 3]

In [39]:
mat2

array([[2, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 1]])

In [40]:
tmp = np.insert(mat2, 1, 0, axis = 0) #index, value to fill, axis 
print(tmp) 

[[2 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 3 0 1]]


In [41]:
tmp = np.insert(tmp, 6, 0, axis = 1)
print(tmp)  

[[2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 3 0 1 0]]
